In [1]:
from datasets import load_dataset
from transformers import BertTokenizerFast
import transformers


trainFile="Data/s288c_words_4.train.txt"



In [2]:
from transformers import BertTokenizer
from transformers import AutoTokenizer


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:

tokenizer_checkpoint = "./S288C_TOKENIZER"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)

In [6]:
model_checkpoint = "bert-base-uncased"


In [7]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [8]:
datasets = load_dataset("text", data_files={"train": trainFile, "validation": testFile})
print(datasets)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
print(tokenized_datasets)


Using custom data configuration default-dc33b54d44e8b545


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /home/lu/.cache/huggingface/datasets/text/default-dc33b54d44e8b545/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 990
    })
})
        

...

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (583 > 512). Running this sequence through the model will result in indexing errorsToken indices sequence length is longer than the specified maximum sequence length for this model (583 > 512). Running this sequence through the model will result in indexing errors

Token indices sequence length is longer than the specified maximum sequence length for this model (828 > 512). Running this sequence through the model will result in indexing errorsToken indices sequence length is longer than the specified maximum sequence length for this model (597 > 512). Running this sequence through the model will result in indexing errors



DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 990
    })
})


In [9]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

# block_size = tokenizer.model_max_length
block_size = 128

tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

print(tokenized_datasets)
print(tokenized_datasets['train'][1])

Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-dc33b54d44e8b545/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-ebbeea374bfd7034.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-dc33b54d44e8b545/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-a57b0d710dce1d64.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-dc33b54d44e8b545/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-869c21f6ad713186.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-dc33b54d44e8b545/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-0f1901b2a0376d64.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-dc33b54d44e8b545/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-56d07f225517ca09.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-dc33b54d44e8b545/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-8af68bf1f7965939.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-dc33b54d44e8b545/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-9c3c867a472bfe7f.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-dc33b54d44e8b545/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-8e768c9016f5634b.arrow


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 990
    })
})
{'input_ids': [2, 67, 53, 36, 124, 27, 143, 24, 94, 22, 56, 22, 37, 188, 59, 83, 230, 22, 110, 83, 28, 85, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [10]:
lm_datasets =tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)



#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    f"CUBERT",
    evaluation_strategy = "epoch",
    learning_rate=1e-3,
    weight_decay=0.01,
    num_train_epochs=1,
)

from transformers import AutoConfig, AutoModelForCausalLM
model_checkpoint = "bert-base-uncased"

config = AutoConfig.from_pretrained(model_checkpoint)

model = AutoModelForCausalLM.from_config(config)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


In [12]:
from bertviz.transformers_neuron_view import BertModel, BertTokenizer
from bertviz.neuron_view import show
from transformers import AutoConfig, AutoModelForMaskedLM

model_type = 'bert'


config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_config(config)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/lu/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [13]:
trainer.train()

/home/lu/.local/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6274
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 785


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
trainer.save_model("CUBERT")